# Segmenting and Clustering Neighborhoods in Toronto - Analysis

### Import libraries

In [63]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1.Visualization of neighborhoods

### Get the latitude and longitude values of New York City using geopy

In [64]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


### Load neighborhoods from previous notebook from CSV

In [65]:
dfNH=pd.read_csv('CAN_neighborhoods_coords.csv')
dfNH.drop(['Unnamed: 0'], axis=1, inplace=True)
print('Resulting shape: ', dfNH.shape)
dfNH.head()

Resulting shape:  (103, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Create a map of Toronto with neighborhoods superimposed on top.

In [66]:
# create map of New York using latitude and longitude values from above
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfNH['Latitude'], dfNH['Longitude'], dfNH['Borough'], dfNH['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods containing the word Toronto. So let's slice the original dataframe and create a new dataframe.

In [67]:
dfT = dfNH[dfNH['Borough'].str.lower().str.contains("toronto")].reset_index(drop=True)
print('Resulting shape: ', dfT.shape)
dfT.head()

Resulting shape:  (39, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


As we did with all of Toronto, let's visualize the inner circle of the neighborhoods in it.

In [68]:
# create map of New York using latitude and longitude values from above
map_Tic = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfT['Latitude'], dfT['Longitude'], dfT['Borough'], dfT['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Tic)  
    
map_Tic

## 2. Explore distance clustering of neighborhoods in Toronto inner circle

### Prepare data for cluster algorithm

In [69]:
dfNH_clustering = dfNH.drop(['Postcode','Borough', 'Neighborhood'], axis=1)
dfNH_clustering.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


### Find the clusters based on the coordinates

In [70]:
# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfNH_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

### Merge the cluster IDs to neighborhoods

In [71]:
dfNH.insert(0, 'Cluster label', kmeans.labels_)
dfNH.head()

,Cluster label,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,0,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,0,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Visualization of the result

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfNH['Latitude'], dfNH['Longitude'], dfNH['Neighborhood'], dfNH['Cluster label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters